In [1]:
#load all the data
import pandas as pd
stories=pd.DataFrame()
comments=pd.DataFrame()
topics=["tamazight","sport","societe","regions","politique","orbites","medias","marocains-du-monde","faits-divers","economie","art-et-culture"]
for topic in topics:
    stories=pd.concat([stories,pd.read_csv("stories_"+topic+".csv")])
    comments=pd.concat([comments,pd.read_csv("comments_"+topic+".csv")])
comments.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
stories.drop(columns=["Unnamed: 0"],axis=1,inplace=True)

## Sentimental Analysis

In [2]:
eng=pd.read_csv("hespress_comments_en.csv")
#lets sample 1000 comments
commentsSample=eng.sample(10000,random_state=20)

In [3]:
commentsSample["en"]

3844     What did the artist offer to the country and w...
45451    We read in your pulpit a few weeks ago that th...
15531    By God Almighty, I shed tears. I am longing fo...
18223    La vie est le reflet de ce que tu crois et pen...
46926    My question is for the specialists. Does the v...
                               ...                        
67344    The immigrant pays the high and precious price...
32106    If everything is reduced, Messi gives everythi...
35608    I am surprised at these nine children who left...
29670    There is no power but from God. Oh God, this evil
1985     To God and to Him we return to the survival of...
Name: en, Length: 10000, dtype: object

In [4]:
#getting arabic sentiment

from camel_tools.sentiment import SentimentAnalyzer
sa = SentimentAnalyzer.pretrained()
predArabic= sa.predict(commentsSample.comment)


I1024 18:24:39.559741  3436 file_utils.py:39] PyTorch version 1.6.0 available.
I1024 18:24:43.860897  3436 configuration_utils.py:262] loading configuration file C:\Users\tariq\AppData\Roaming\camel_tools\data\sentiment_analysis\arabert\config.json
I1024 18:24:43.861899  3436 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "eos_token_ids": null,
  "finetuning_task": "arabic_sentiment",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "out

In [5]:
# commentsSample["positive"] =list(map(lambda prediction: prediction["positive"],predArabic))
# commentsSample["negative"] = list(map(lambda prediction: prediction["negative"],predArabic))
# commentsSample["neutral"] =list(map(lambda prediction: prediction["neutral"],predArabic))

# commentsSample["positive"] =list(map(lambda prediction: prediction["positive"],predArabic))
# commentsSample["negative"] = list(map(lambda prediction: prediction["negative"],predArabic))
commentsSample["sentimentArabic"]= predArabic

In [6]:
# commentsSample.

SyntaxError: invalid syntax (<ipython-input-6-bba6a9d0c522>, line 1)

In [7]:
import numpy as np
def indexToPred(index):
    preds=["positive","negative","neutral"]
    return preds[index] 

def polariyToSentiment(polarity):
    if polarity>0:
        return "positive"
    elif polarity==0:
        return "neutral"
    else:
        return "negative"

# commentsSample["sentimentArabic"]=commentsSample.apply(lambda comment: indexToPred(np.argmax([comment.positive,comment.negative,comment.neutral])),axis=1)

In [8]:
#Getting english sentiment

from textblob import TextBlob
sentiments=[]
for sentence in commentsSample.en:
    sentiments.append(TextBlob(sentence).sentiment)

In [9]:
commentsSample["sentimentEnglish"]=[polariyToSentiment(sentiment.polarity) for sentiment in sentiments]

In [10]:
#counting how much they resamble
commentsSample.apply(lambda comment: comment.sentimentEnglish==comment.sentimentArabic,axis=1).sum()

3951

Out of 10000 comments only 3951 are similar between Camel Tools and Text Blob using translated text


In [11]:
10000-3951

#60%

6049

In [14]:
from sklearn.metrics import classification_report
print(classification_report(commentsSample.sentimentEnglish,commentsSample.sentimentArabic))

              precision    recall  f1-score   support

    negative       0.30      0.83      0.44      2336
     neutral       0.43      0.22      0.29      2337
    positive       0.65      0.28      0.39      5327

    accuracy                           0.40     10000
   macro avg       0.46      0.44      0.37     10000
weighted avg       0.52      0.40      0.38     10000



In [82]:
pd.set_option('display.max_colwidth', -1)
commentsSample.iloc[5][["comment","en","sentimentArabic","sentimentEnglish"]]

comment             مغادي نهضر على حتى شي حاجة عندها علاقة بان هاد الوباء عطى و بين المكانة اللي كيستحق كل واحد\nولكن الحاجة اللي غادي نعلق عليها هي كلمة النجمين لا اظن ان كلمة نجم عدنا فالمغرب                                                                                      
en                  I will go out, until something needs to be done. It has a relationship that the head of the epidemic is given and the status that everyone deserves\nBut the need that we attach to is the word al-najmeen. I don’t think the word “star” is our return, so Morocco
sentimentArabic     negative                                                                                                                                                                                                                                                           
sentimentEnglish    neutral                                                                                                                                     

## Named Entity Recognition

In [3]:
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

# Predict the labels of a single sentence.
# The sentence must be pretokenized by whitespace and punctuation.


sentence = 'سمير ذهب الى مراكش لشراء هاتف سامسونغ.'.split()

labels = ner.predict_sentence(sentence)

# Print the list of token-label pairs
print(list(zip(sentence, labels)))

I1022 08:13:09.029264 16104 file_utils.py:39] PyTorch version 1.6.0 available.
I1022 08:13:13.295162 16104 configuration_utils.py:262] loading configuration file C:\Users\tariq\AppData\Roaming\camel_tools\data\ner\arabert\config.json
I1022 08:13:13.297128 16104 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-MISC",
    "2": "B-ORG",
    "3": "B-PERS",
    "4": "I-LOC",
    "5": "I-MISC",
    "6": "I-ORG",
    "7": "I-PERS",
    "8": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-MISC": 1,
    "B-ORG": 2,
    "B-PERS": 3,
    "I-LOC": 4,
    "I-MISC": 5,
    "I-ORG": 6,
    "I-PERS": 7,
    "O": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_

[('سمير', 'B-PERS'), ('ذهب', 'O'), ('الى', 'O'), ('مراكش', 'B-LOC'), ('لشراء', 'O'), ('هاتف', 'O'), ('سامسونغ.', 'B-ORG')]


## Tagging

In [4]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')

tagger.tag('ذهبت الى المدرسة'.split())

['verb', 'prep', 'noun']

In [5]:
tagger = DefaultTagger(mled, 'diac')

tagger.tag('ذهبت الى المدرسة'.split())

['ذَهَبْتُ', 'إِلَى', 'المَدْرَسَةِ']

In [6]:
tagger = DefaultTagger(mled, 'num')
tagger.tag('ذهبت الى المدارس'.split())

['s', 'na', 'p']

In [7]:
tagger = DefaultTagger(mled, 'form_gen')
tagger.tag('اكل الولد و الفتاة التفاحة'.split())

['m', 'm', 'na', 'f', 'f']

In [39]:
#lemmatization

tagger = DefaultTagger(mled, 'lex')
tagger.tag('اكل الولد و الفتاة التفاحة'.split())

['أَكَل-u_1', 'وَلَد_1', 'وَ_1', 'فَتاة_1', 'تُفّاح_1']

In [8]:
from camel_tools.disambig.mle import MLEDisambiguator

mle = MLEDisambiguator.pretrained()

# We expect a sentence to be whitespace/punctuation tokenized beforehand.
# We provide a simple whitespace and punctuation tokenizer as part of camel_tools.
# See camel_tools.tokenizers.word.simple_word_tokenize.
sentence = ['سوف', 'نقرأ', 'الكتب']

disambig = mle.disambiguate(sentence)

# Let's, for example, use the top disambiguations to generate a diacritized
# version of the above sentence.
# Note that, in practice, you'll need to make sure that each word has a
# non-zero list of analyses.
diacritized = [d.analyses[0].analysis['diac'] for d in disambig]
print(' '.join(diacritized))

سَوْفَ نَقْرَأ الكُتُبِ


## Dialect Identification

In [9]:
from camel_tools.dialectid import DialectIdentifier

did = DialectIdentifier.pretrained()

sentences = [
    'مال الهوى و مالي شكون اللي جابني ليك  ما كنت انايا ف حالي بلاو قلبي يانا بيك',
    'بدي دوب قلي قلي بجنون بحبك انا مجنون ما بنسى حبك يوم'
]

predictions = did.predict(sentences)

# Each prediction is a tuple containing both the top prediction and the
# percentage score of each dialect. To get only the top prediction, we can
# do the following:
top_dialects = [p.top for p in predictions]

ModuleNotFoundError: camel_tools.dialectid is not available on Windows.

In [38]:
from camel_tools.disambig.mle import MLEDisambiguator

mle = MLEDisambiguator.pretrained()

# We expect a sentence to be whitespace/punctuation tokenized beforehand.
# We provide a simple whitespace and punctuation tokenizer as part of camel_tools.
# See camel_tools.tokenizers.word.simple_word_tokenize.
sentence = ['سوف', 'نقرأ', 'الكتب',"يسعدني"]

disambig = mle.disambiguate(sentence)

# Let's, for example, use the top disambiguations to generate a diacritized
# version of the above sentence.
# Note that, in practice, you'll need to make sure that each word has a
# non-zero list of analyses.
diacritized = [d.analyses[0].analysis['lex'] for d in disambig]
print(' '.join(diacritized))

سَوْفَ_1 قَرَأ-a_1 كِتاب_1 أَسْعَد_1


DisambiguatedWord(word='الكتب', analyses=[ScoredAnalysis(score=1.0, analysis={'diac': 'الكُتُبِ', 'lex': 'كِتاب_1', 'bw': 'ال/DET+كُتُب/NOUN+ِ/CASE_DEF_GEN', 'gloss': 'the+books+[def.gen.]', 'pos': 'noun', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': 'Al_det', 'per': 'na', 'asp': 'na', 'vox': 'na', 'mod': 'na', 'stt': 'd', 'cas': 'g', 'enc0': '0', 'rat': 'i', 'source': 'lex', 'form_gen': 'm', 'form_num': 's', 'num': 'p', 'd2seg': 'الكُتُبِ', 'd1tok': 'الكُتُبِ', 'root': 'ك.ت.ب', 'd3tok': 'ال+_كُتُبِ', 'caphi': '2_a_l_k_u_t_u_b_i', 'pos_lex_logprob': -3.511249, 'd1seg': 'الكُتُبِ', 'ud': 'DET+NOUN', 'd2tok': 'الكُتُبِ', 'lex_logprob': -3.511249, 'atbtok': 'الكُتُبِ', 'gen': 'm', 'd3seg': 'ال+_كُتُبِ', 'catib6': 'PRT+NOM', 'atbseg': 'الكُتُبِ', 'pattern': 'ال1ُ2ُ3ِ', 'pos_logprob': -0.4344233, 'stem': 'كُتُب', 'stemgloss': 'books', 'stemcat': 'N'})])